<a href="https://colab.research.google.com/github/AngelBReal/Proyecto_IC_MCD_GINI/blob/agrega-raw/notebooks/tidy_ENCO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Proyecto 1. Descargando datos de la web***

---

Curso Ingeniería de Características


Libreta con código para pasar a formaro tidy los datos de la Encuesta Nacional sobre Confianza del Consumidor (ENCO) en el año 2022, para su posterior comparación con los datos de la Encuesta Nacional de Ingresos y Gastos de los Hogares (ENIGH) en el mismo año.

In [1]:
# Importar librerías necesarias
import requests
import zipfile
import os
import pandas as pd
from io import BytesIO

# URLs de los archivos a descargar (ajustar según sea necesario)
base_url = "https://www.inegi.org.mx/contenidos/programas/enco/datosabiertos/2022/"
urls = [f"{base_url}conjunto_de_datos_enco_2022_{str(i).zfill(2)}_csv.zip" for i in range(1, 13)]

# Función para descargar y extraer archivos zip
def descargar_y_extraer_zip(url, extract_path='/content/enco_2022'):
    response = requests.get(url)
    if response.status_code == 200:
        with zipfile.ZipFile(BytesIO(response.content)) as z:
            z.extractall(extract_path)
            print(f"Archivos extraídos de {url}")
    else:
        print(f"Error al descargar {url}")

# Descarga y extracción de los archivos en la carpeta enco_2022
os.makedirs('/content/enco_2022', exist_ok=True)

for url in urls:
    descargar_y_extraer_zip(url)

# Definir las columnas comunes y específicas para cada conjunto de datos
columnas_comunes = ['fol', 'ent', 'con', 'v_sel', 'n_hog', 'h_mud']

# Columnas específicas para cada archivo
viv_especificas = ['ageb', 'fch_def']
cs_especificas = ['i_per', 'ing']
cb_especificas = [f'p{i}' for i in range(1, 16)]  # 'p1' a 'p15' para 'cb'

# Definir las columnas que se usarán en cada conjunto de datos
viv_cols = columnas_comunes + viv_especificas
cs_cols = columnas_comunes + cs_especificas
cb_cols = columnas_comunes + cb_especificas

# Función para seleccionar las columnas relevantes
def seleccionar_columnas(df, columnas_relevantes):
    return df[columnas_relevantes]

# Leer y filtrar los DataFrames de cada archivo
base_path = '/content/enco_2022'

# Función para leer y almacenar los DataFrames
def cargar_datos(mes, tipo):
    file_name = f'conjunto_de_datos_{tipo}_enco_2022_{str(mes).zfill(2)}.CSV'
    folder_path = f'{base_path}/conjunto_de_datos_{tipo}_enco_2022_{str(mes).zfill(2)}/conjunto_de_datos'
    file_path = os.path.join(folder_path, file_name)

    if os.path.exists(file_path):
        return pd.read_csv(file_path)
    else:
        print(f"Archivo no encontrado: {file_name}")
        return pd.DataFrame()  # Devolver DataFrame vacío si no se encuentra el archivo

# Almacenar los DataFrames filtrados para cada conjunto
cs_enco_filtrado = [seleccionar_columnas(cargar_datos(i, 'cs'), cs_cols) for i in range(1, 13)]
viv_enco_filtrado = [seleccionar_columnas(cargar_datos(i, 'viv'), viv_cols) for i in range(1, 13)]
cb_enco_filtrado = [seleccionar_columnas(cargar_datos(i, 'cb'), cb_cols) for i in range(1, 13)]

# Unir los DataFrames por las columnas comunes y concatenar los resultados
df_final = pd.DataFrame()

for cs_df, viv_df, cb_df in zip(cs_enco_filtrado, viv_enco_filtrado, cb_enco_filtrado):
    # Solo hacer merge si los DataFrames no están vacíos
    if not cs_df.empty and not viv_df.empty and not cb_df.empty:
        temp = pd.merge(cs_df, viv_df, on=columnas_comunes, how='inner')
        temp = pd.merge(temp, cb_df, on=columnas_comunes, how='inner')
        df_final = pd.concat([df_final, temp], ignore_index=True)

# Visualizar las primeras filas del DataFrame final
df_final.head()

# Guardar el DataFrame tidy procesado
df_final.to_csv('/content/enco_2022/processed_enco_tidy.csv', index=False)


Archivos extraídos de https://www.inegi.org.mx/contenidos/programas/enco/datosabiertos/2022/conjunto_de_datos_enco_2022_01_csv.zip
Archivos extraídos de https://www.inegi.org.mx/contenidos/programas/enco/datosabiertos/2022/conjunto_de_datos_enco_2022_02_csv.zip
Archivos extraídos de https://www.inegi.org.mx/contenidos/programas/enco/datosabiertos/2022/conjunto_de_datos_enco_2022_03_csv.zip
Archivos extraídos de https://www.inegi.org.mx/contenidos/programas/enco/datosabiertos/2022/conjunto_de_datos_enco_2022_04_csv.zip
Archivos extraídos de https://www.inegi.org.mx/contenidos/programas/enco/datosabiertos/2022/conjunto_de_datos_enco_2022_05_csv.zip
Archivos extraídos de https://www.inegi.org.mx/contenidos/programas/enco/datosabiertos/2022/conjunto_de_datos_enco_2022_06_csv.zip
Archivos extraídos de https://www.inegi.org.mx/contenidos/programas/enco/datosabiertos/2022/conjunto_de_datos_enco_2022_07_csv.zip
Archivos extraídos de https://www.inegi.org.mx/contenidos/programas/enco/datosabier